In [6]:
from chaotic_inventory_opt.control.classical import sSPolicy, BaseStockPolicy
from chaotic_inventory_opt.control.fcio import FCIOPolicy
from chaotic_inventory_opt.control.network import NetworkFCIOPolicy

from chaotic_inventory_opt.core.inventory_system import InventorySystem
from chaotic_inventory_opt.core.cost import CostModel
from chaotic_inventory_opt.evaluation.performance import PerformanceMetrics
from chaotic_inventory_opt.evaluation.stability import StabilityMetrics
from chaotic_inventory_opt.chaos.lyapunov import LyapunovExponentEstimator

import pandas as pd
import numpy as np
import optuna
import multiprocessing
from concurrent.futures import ThreadPoolExecutor


In [7]:
optuna.logging.set_verbosity(optuna.logging.INFO)


In [8]:
cost_model = CostModel(
    holding_cost_per_unit=0.1,
    stockout_cost_per_unit=5.0,
    fixed_order_cost=1.0
)


In [9]:
def load_m5_demand(item_id, store_id):
    sales = pd.read_csv("sales_train_validation.csv")
    row = sales[
        (sales["item_id"] == item_id) &
        (sales["store_id"] == store_id)
    ]
    d_cols = [c for c in row.columns if c.startswith("d_")]
    return row[d_cols].values.flatten().astype(float)



In [10]:
sales = pd.read_csv("sales_train_validation.csv")
DEMAND_COLS = [c for c in sales.columns if c.startswith("d_")]

demand_matrix = sales[DEMAND_COLS].values.astype(np.float64)
N_SKUS, T_TOTAL = demand_matrix.shape

print("SKUs:", N_SKUS, "Timesteps:", T_TOTAL)


SKUs: 30490 Timesteps: 1913


In [11]:
def temporal_split(mat, ratio=0.7):
    split = int(mat.shape[1] * ratio)
    return mat[:, :split], mat[:, split:]


demand_train, demand_test = temporal_split(demand_matrix)

print("Train shape:", demand_train.shape)
print("Test shape:", demand_test.shape)


Train shape: (30490, 1339)
Test shape: (30490, 574)


In [12]:
np.random.seed(42)

SUBSET_SIZE = 300 
TUNE_HORIZON = 300 

subset_idx = np.random.choice(N_SKUS, SUBSET_SIZE, replace=False)
train_subset = demand_train[subset_idx, :TUNE_HORIZON]


In [22]:
def simulate_subset(policy_factory, demand_subset, trial=None):
    N, T = demand_subset.shape

    systems = [InventorySystem(30) for _ in range(N)]
    policies = [policy_factory() for _ in range(N)]

    perf = PerformanceMetrics()
    stab = StabilityMetrics(LyapunovExponentEstimator())

    for t in range(T):
        demands = demand_subset[:, t]

        for i in range(N):
            if hasattr(policies[i], "observe"):
                policies[i].observe(demands[i])

        for i in range(N):
            q = policies[i].order(systems[i].I)
            inv = systems[i].step(demands[i], q)
            cost = cost_model.compute(inv, q)

            perf.update(demands[i], inv, cost)
            stab.update(inv)

        if trial is not None and t % 100 == 0 and t > 0:
            trial.report(perf.total_cost, step=t)
            if trial.should_prune():
                raise optuna.TrialPruned()

    return perf.results(), stab.results()


In [14]:
def fcio_objective(trial):
    print(f"Running trial {trial.number}")

    params = dict(
        alpha=trial.suggest_float("alpha", 0.2, 1.2),
        beta=trial.suggest_float("beta", 0.2, 1.2),
        scale_min=trial.suggest_float("scale_min", 0.5, 0.8),
        scale_max=trial.suggest_float("scale_max", 1.2, 1.6),
        rho=trial.suggest_float("rho", 0.5, 0.75),
        gamma=trial.suggest_float("gamma", 1.2, 1.8),
        k_cap=trial.suggest_float("k_cap", 0.5, 1.0),
    )

    def policy_factory():
        return FCIOPolicy(
            base_stock_level=30,
            reorder_point=10,
            window=50,
            **params
        )

    results, _ = simulate_subset(
        policy_factory,
        train_subset,
        trial=trial
    )

    score = (
        results["total_cost"]
        + 3_000 * results["stockout_events"]
        + 100_000 * max(0, 0.95 - results["service_level"])
    )

    return score


In [15]:
pruner = optuna.pruners.MedianPruner(
    n_startup_trials=2,
    n_warmup_steps=1,
    interval_steps=1
)

study = optuna.create_study(
    direction="minimize",
    pruner=pruner
)

study.optimize(
    fcio_objective,
    n_trials=15,
    n_jobs=min(6, multiprocessing.cpu_count()),
    show_progress_bar=True
)

print("Best parameters:", study.best_params)


[I 2026-01-02 23:56:31,508] A new study created in memory with name: no-name-5da6da7d-f309-4fd4-a3a6-5c2c2bdb5257
  0%|          | 0/15 [00:00<?, ?it/s]

Running trial 0
Running trial 1
Running trial 2
Running trial 3
Running trial 4
Running trial 5


  0%|          | 0/15 [03:15<?, ?it/s]

[I 2026-01-02 23:59:46,021] Trial 3 finished with value: 8000118.138008195 and parameters: {'alpha': 0.3399903028499697, 'beta': 1.1218620043232914, 'scale_min': 0.6301951011993461, 'scale_max': 1.2921079784150657, 'rho': 0.5125813790441024, 'gamma': 1.586333277370715, 'k_cap': 0.8436690138360011}. Best is trial 3 with value: 8000118.138008195.


Best trial: 3. Best value: 8.00012e+06:   7%|▋         | 1/15 [03:15<45:40, 195.74s/it]

...

[I 2026-01-03 00:03:25,341] Trial 9 finished with value: 2693758.0585277393 and parameters: {'alpha': 1.101634759404177, 'beta': 0.37488462812492623, 'scale_min': 0.7706894566011648, 'scale_max': 1.3726558955039974, 'rho': 0.6880253100857288, 'gamma': 1.7431912155688583, 'k_cap': 0.6244534843862106}. Best is trial 9 with value: 2693758.0585277393.


Best trial: 9. Best value: 2.69376e+06:  93%|█████████▎| 14/15 [06:58<00:16, 16.97s/it]

[I 2026-01-03 00:03:30,007] Trial 11 finished with value: 3283913.350985481 and parameters: {'alpha': 0.5646447769729848, 'beta': 0.4041305641326632, 'scale_min': 0.6429895746979121, 'scale_max': 1.3347703231233343, 'rho': 0.6939468645559732, 'gamma': 1.6744005195507135, 'k_cap': 0.8297866983548072}. Best is trial 9 with value: 2693758.0585277393.


Best trial: 9. Best value: 2.69376e+06: 100%|██████████| 15/15 [07:05<00:00, 28.36s/it]

[I 2026-01-03 00:03:36,943] Trial 13 finished with value: 3245861.7133409805 and parameters: {'alpha': 0.536404970744309, 'beta': 1.1821466306994346, 'scale_min': 0.755868828933052, 'scale_max': 1.5167496693073426, 'rho': 0.5867242786267728, 'gamma': 1.7781215723581996, 'k_cap': 0.6986018062805347}. Best is trial 9 with value: 2693758.0585277393.
Best parameters: {'alpha': 1.101634759404177, 'beta': 0.37488462812492623, 'scale_min': 0.7706894566011648, 'scale_max': 1.3726558955039974, 'rho': 0.6880253100857288, 'gamma': 1.7431912155688583, 'k_cap': 0.6244534843862106}


In [18]:
TEST_SUBSET_SIZE = 1000      # start with 1000, not 2000
TEST_HORIZON = 300           # not full horizon

np.random.seed(2026)

test_idx = np.random.choice(N_SKUS, TEST_SUBSET_SIZE, replace=False)
demand_test_eval = demand_test[test_idx, :TEST_HORIZON]

print("Test SKUs:", demand_test_eval.shape[0])
print("Test horizon:", demand_test_eval.shape[1])


Test SKUs: 1000
Test horizon: 300


In [19]:
results_fcio, stab_fcio = simulate_subset(
    best_policy_factory,
    demand_test_eval
)


In [20]:
print("FCIO TEST (ALL SKUs):", results_fcio, stab_fcio)

FCIO TEST (ALL SKUs): {'total_cost': 979735.2932534337, 'service_level': 0.9547583720301375, 'stockout_events': 4281} {'inventory_variance': 37.342184661777154, 'lyapunov_exponent': 1.5785162581580343}


In [23]:
def ss_factory():
    return sSPolicy(10, 30)

def bs_factory():
    return BaseStockPolicy(30)

results_ss, stab_ss = simulate_subset(ss_factory, demand_test)
results_bs, stab_bs = simulate_subset(bs_factory, demand_test)

print("TEST (s,S):", results_ss, stab_ss)
print("TEST Base Stock:", results_bs, stab_bs)


Exception ignored in: <generator object _no_nep50_warning at 0x00000221806727A0>
Traceback (most recent call last):
  File "c:\Users\SANTANU\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\_ufunc_config.py", line 466, in _no_nep50_warning
    NO_NEP50_WARNING.reset(token)
ValueError: <Token var=<ContextVar name='_no_nep50_warning' default=False at 0x00000221FE75E6B0> at 0x00000221A7E3C980> was created in a different Context


TEST (s,S): {'total_cost': 40923097.99981631, 'service_level': 0.9512678043770195, 'stockout_events': 89275} {'inventory_variance': 52.518400000000014, 'lyapunov_exponent': 0.9624905771473374}
TEST Base Stock: {'total_cost': 60735661.496415704, 'service_level': 0.967573147557619, 'stockout_events': 38499} {'inventory_variance': 22.432399999999998, 'lyapunov_exponent': -3.401194665078935}


In [24]:
pd.DataFrame([
    {**results_ss, **stab_ss, "policy": "(s,S)"},
    {**results_bs, **stab_bs, "policy": "Base Stock"},
    {**results_fcio, **stab_fcio, "policy": "FCIO (generalized)"},
]).set_index("policy")


,total_cost,service_level,stockout_events,inventory_variance,lyapunov_exponent
policy,,,,,
"(s,S)",4.092310e+07,0.951268,89275,52.518400,0.962491
Base Stock,6.073566e+07,0.967573,38499,22.432400,-3.401195
FCIO (generalized),9.797353e+05,0.954758,4281,37.342185,1.578516
